In [ ]:
from imp import reload
import utils; reload(utils); from utils import *
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import operator
from sklearn.manifold import TSNE
from sklearn.feature_selection import *
from sklearn import metrics
from xgboost import XGBClassifier, plot_importance, plot_tree
from collections import Counter
import xgboost
import pickle
from __future__ import division, print_function
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import auc, roc_curve
from sklearn.decomposition import PCA
from itertools import combinations
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from imblearn import under_sampling
from sklearn.ensemble.bagging import BaggingClassifier

In [ ]:
def preprocess_data(data, training=True):
    if training:
        y_train = data.flag
        data.drop(['flag', 'no'], axis=1, inplace=True)
    else:
        pred_no = data['no']
        data.drop(['no'], axis=1, inplace=True)
        
    data.columns = map(str.lower, data.columns)
    
    mm_scale = MinMaxScaler()
    data['num_of_not_null'] = np.sum(data.isnull(), axis=1)
    data = data.fillna(0)
    data['row_var'] = data.var(axis=1)
#     data['row_sum'] = data.sum(axis=1)
#     data['row_max'] = data.max(axis=1)
    data['row_min'] = data.min(axis=1)

    features_raw = data.columns.values.tolist()
    data = mm_scale.fit_transform(data)
    data = np.log(data+1) 
    data = pd.DataFrame(data, columns=features_raw)
    if training == False:
        return data, pred_no, features_raw
    return data, y_train, features_raw

def modelfit(alg, dtrain, target, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
#         xgtrain = xgboost.DMatrix(dtrain.values, label=target.values)
        xgtrain = xgboost.DMatrix(dtrain, label=target)
        cvresult = xgboost.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], 
                              stratified=True, nfold=cv_folds, metrics='auc', 
                              early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    # Fit the algorithm on the data
    alg.fit(dtrain, target, eval_metric='auc')
        
    # Predict training set:
    dtrain_predictions = alg.predict_proba(dtrain)
    roc_result = roc_auc_score(target, dtrain_predictions[:,1])
    #Print model report:
    print("\nModel Report")
#     print(cvresult.tail(1)['test-auc-mean'].values[0])
    print(cvresult.tail(1))
    print("auc: %.4f" % roc_result)
    print("Optimal number of trees (estimators) is %i" % cvresult.shape[0])    
    f = plt.figure(figsize=[10, 12])
    ax = f.add_subplot(111)
    plot_importance(alg, ax=ax)

    return cvresult.tail(1)['test-auc-mean'].values[0], roc_result

In [ ]:
X_train_A, y_A, features = preprocess_data(pd.read_csv('data/A_train.csv'))
X_train_B, y_B, features = preprocess_data(pd.read_csv('data/B_train.csv'))
X_test_B, pred_no, features = preprocess_data(pd.read_csv('data/B_test.csv'), training=False)
xgb = XGBClassifier(scale_pos_weight=3)
xgb.fit(X_train_A, y_A)
X_train_B_N = pd.concat((X_train_B, pd.DataFrame(xgb.predict_proba(X_train_B)[:,1])), axis=1)
X_test_B_N = pd.concat((X_test_B, pd.DataFrame(xgb.predict_proba(X_test_B)[:,1])), axis=1)

In [ ]:
xgb1.set_params(n_estimators=103)
xgb1.set_params(max_depth=2)
xgb1.set_params(min_child_weight=4)
xgb1.set_params(colsample_bytree=0.74)
xgb1.set_params(subsample=0.91)
xgb1.set_params(reg_alpha=0.007)
xgb1.set_params(learning_rate=0.01)

In [ ]:
filename = "data/predict_from_A.csv"
bg_a_plus_b = BaggingClassifier(xgb1, n_estimators=50)
bg_a_plus_b.fit(X_train_B_N, y_B)
preds = bg_a_plus_b.predict_proba(X_test_B_N)
submit(pred_no, preds, filename)